# Communicating Our Findings in Maji Ndogo

### Introduction

This project is focused on providing decision-makers with accurate, actionable data regarding water access in Maji Ndogo, particularly for national and provincial levels. The goal is to create a report that presents the analysis results and key insights clearly so that stakeholders can make informed decisions about water access improvements and necessary funding.

## User Story

When reviewing the report from our previous project, it's been challenging to understand how the data fits together.

To design an effective report, we use a "user story" framework. This framework helps us put ourselves in the shoes of the end users to understand their needs and how they will interact with the report. 

### National Level Dashboard Overview

Let’s approach this from the perspective of one of our users. Picture yourself as the company director, and we are aiming to:
1. See the key points of the survey results to understand the overall status of water access in Maji Ndogo.
2. Know how many people are affected by water access challenges and what those challenges are.
3. Understand how much money is needed to complete the upgrades, and where that money needs to be spent.
4. View and understand this data at both the national and provincial levels.

Let’s begin with our director's page, which is structured around three key components: the population/water breakdown, "What do we need to do?" and "How much will it cost?". To effectively present this information, we will split the dashboard into these three sections while also reserving space for slicers to enable dynamic filtering of data. Additionally, we will allocate space to highlight the big idea results that directly address the core purpose: "How much do we need to spend in Maji Ndogo, and what will that money do?"

This framework is a starting point for our visualisations, we’ll continue to adjust and refine the visuals based on the data and insights we gather.

![Dashboard](water_access/visualisation/part7/assets/national_dashboard.png)

I grouped similar things in boxes, and the three different boxes tell this story:
- Who are we doing this for and where are they?
- What do we need to do, and where do we need to do it?
- What will this cost?

The flow of this data story follows a logical progression. We start by understanding who we are improving water access for, then we identify the areas where improvements are needed, and finally, we address the associated costs. This order makes sense, right?

Next, for each of these user stories, we have to think about **what data** the user will need to answer that question and **how** will we visualise that data.

So let's apply the what and the how.
1. See the key points of the survey results to understand the overall status of water access in Maji Ndogo.
    - We should summarise population-related access to water on a national and provincial level.
    - A lot of people in Maji Ndogo live in rural and urban areas, so we should summarise their results and the challenges they face separately by splitting the page into rural vs urban.

2. Know how many people are affected by water access challenges in Maji Ndogo and what those challenges are.
    - We need to communicate the number of sources, and also the amount of people affected by these sources by showing the distribution of source types in rural vs. urban populations.
    - Perhaps we should also consider making the less important sources like tap_in_homes less visible since we're not upgrading those.

3. View and understand how much money we will need to complete the upgrades, and where that money needs to be spent.
     - We need to show a single number representing the data nationally, and per province. There is some financial data in our data model called `infrastructure_cost`, but we will need to do some calculations to include this type of data. We will need to see this data at a national, provincial, and rural/urban level, and even per `improvement` type.

4. View and understand this data on a national level and a provincial level.
    - We have to take this into account, as well as at a rural/urban split. If we build the report well, we can drill down into different categories.

We will create a new visuals page, National, a title, and start adding visuals! We will also, add a slicer to the controls section so that we can select provinces. I also added a map. So now, when we select a province in the slicer, the map highlights where that province is.

![National Report](water_access/visualisation/part7/assets/progress_report.png)

## Data Transformation and Analysis Using DAX

Up until now, we've used the data in its raw form. However, to create more meaningful and impactful visuals, we performed some calculations and summarized the data to enhance the insights.

### 1. Aggregating Improvement Categories

Using **DAX**, `improvement` categories such as `Install 1-8 taps` were consolidated into `Install public tap(s)*` and `Diagnose local infrastructure` was shortened too. The * in `Install public tap(s)*` is because the meaning of this category isn't quite the same now that we've aggregated it. A value of 3696 for "Install public tap(s)" means the total count of all these installations combined, but it doesn’t tell us how many of those were for 1 tap, 2 taps, or more.

> **NOTE**: The category `Install public tap(s)*` combines all the data from the "Install 1-8 taps" categories into one total. This makes the chart simpler but loses the details about how many taps were installed in each case (like 1 tap, 2 taps, etc.).

In [ ]:
# DAX Code for Aggregation:

'''Aggregated_improvements = 
IF(
    CONTAINSSTRING('project_progress'[improvement], "Install") &&
    CONTAINSSTRING('project_progress'[improvement], "taps"),
    "Install public tap(s)",
    IF(
        'project_progress'[improvement] == "Diagnose local infrastructure",
        "Repair infrastructure",
        'project_progress'[improvement]
    )
)'''

### 2. Budget Calculations

As expected, the cost of these improvements is significant. To ensure efficient use of resources, we will begin by creating a detailed budget.

The project management team has provided an `infrastructure_cost` table that estimates the costs associated with improving various water sources. Each record in this table represents the cost, in US dollars, to improve a specific water source. For example, drilling a well to enhance a river water source comes with an estimated cost of $8,500.

After further analysis, the project planners determined that improving rural water sources is more challenging and resource-intensive compared to urban sources. To account for this, they recommended increasing the budget for rural water sources by 50%. 

- **Rural Cost Adjustment**: The above adjustment will be captured in a new column called `Rural_adjusted_cost`, which will reflect the updated cost for rural water source improvements.

In [ ]:
# Calculate the increased budget for each of the improvements

'''Rural_adjusted_cost = infrastructure_cost[unit_cost_USD] * 1.5''

- **Improvement Costs**: Calculated budget costs for each improvement, based on the type of improvement and whether the source was rural or urban.

In [ ]:
# Formula that looks up the improvement and either fetches the value

'''Budgeted_improvement_cost divided by Sum of number_of_people_served = 
DIVIDE(
	SUM('project_progress'[Budgeted_improvement_cost]),
	[Total_number_of_people]
)'''

### Creating `Basic_water_access` Column. 

To measure project success, water sources were classified based on UN requirements. In order for a water source to be classified as basic, it must satisfy these requirements:
- Rivers are not improved sources, so they are not included.
- Wells are improved, only if they are clean. So all contaminated wells are excluded.
- Public taps are improved sources, but only if the queue time is less than 30 min.
- Broken infrastructure "taps" are not basic, because they do not work.
- All taps installed in homes across Maji Ndogo are at least basic. They are actually safely managed.

To assess the impact of upgrading water sources, we first need to determine the current level of water access in the area. Our goal is to progressively increase the percentage of people with access to clean water, aiming for full access (100%) as we make improvements. By tracking these changes, we can measure the success of the project and ensure that our efforts are translating into tangible benefits.

So the idea is to calculate the number of clean wells, the number of public taps with queue times < 30 min, and taps in homes, then use that to
calculate how many people have access to basic water. We could do that directly, but including all the conditional logic would introduce unnecessary complexity. Instead, we break it three steps:

1. Classify sources as either "Basic Access" or "Below Basic Access" in the new column. The classification depends on several factors:
- For wells, we need to check if the water is contaminated, which is determined by the `well_pollution` table.
- But to get the queue time of a shared tap, we have to remember that shared taps were visited multiple times, so it is not as simple as fetching the queue time for each row, because there are multiple entries for each source_id.

2. Once we have that, we can sum up all of the people using a basic water source, divided by the total population.

Step 1 is complex, so let's break it down a bit more. 

#### Step A: Calculating the Average Queue Time by Water Source

We will calculate the average queue time for each unique water source to get an overview of the time spent at each location. To do this, we will need to aggregate the data from the `water_source` table based on the `source_id`. 

In [ ]:
# Calculate average queue time filtered by each specific water source visit

'''Average_queue_time = 
    CALCULATE(
        AVERAGE('visits'[time_in_queue]),
        FILTER(
            'visits',
            'visits'[source_id] = 'water_source'[source_id] &&
            'visits'[visit_count] = 1
        )
    )'''

#### Step B: Creating Control Flow Logic for Classifying Wells and Taps

The second task is to create a control flow logic to check wells and taps to classify them as being basic or below basic in the `Basic_water_access` column.

In [ ]:
# Classify each source as being basic or below basic

'''Basic_water_access = 
    IF(
        AND( // Checks if the water source is a well and if its clean
            'water_source'[type_of_water_source] = "well",
            RELATED('well_pollution'[results]) = "Clean"
        ),
        "Basic Access", // Assigns 'Basic Access' if both conditions are true (clean well)
        IF( // Checks if the water source is a tap inside the home
            'water_source'[type_of_water_source] = "tap_in_home",
            "Basic Access", // Assigns 'Basic Access' if true (tap in home)
            IF(
                AND( // Checks if the water source is a shared tap and the average queue time is less than 30 minutes
                    'water_source'[type_of_water_source] = "shared_tap",
                    'water_source'[Average_queue_time] < 30
                    ),
                "Basic Access", // Assigns 'Basic Access' if both conditions are true (shared tap with queue < 30 min)
                "Below Basic Access" // Assigns 'Below Basic Access' if none of the above conditions are met
)))'''

The final step is to create a new measure to calculate the basic water access level. This measure will be the ratio of people with basic water access to the total population in Maji Ndogo. 

In [ ]:
# Percentage population water access

'''Pct_population_access = 
    ROUND (
        DIVIDE (
            'water_source'[Number_of_people_basic_access],
            'water_source'[Total_number_of_people]
        ) * 100,
        0 -- specifier for no decimals
    )'''

And a last-last step, let's consider an important insight! The basic access percentage tells us how many people have access to basic water services, but we might also need the percentage of people who don't have access to basic water too. After all, our ultimate goal would be to reduce this number to 0%, ensuring universal access to basic water services. This perspective helps highlight the gap we need to address to achieve equitable access.

In [ ]:
# Percentage population without water access

'''Pct_population_without_access = 
    ROUND (
        DIVIDE (
            'water_source'[Number_of_people_without_basic_access],
            'water_source'[Total_number_of_people]
        ) * 100,
        0 -- Format specifier for no decimals
    )'''

### Providing Financial Insights for Decision-Making

We aim to equip our directors with the financial insights necessary for informed decision-making. They will want to understand the costs at a provincial level and also understand what the different improvements will cost us.

To do this, we'll create a budget table that provides a detailed breakdown of costs by province and improvement type and add a visual that will show the percentages of the budget allocated to each province.

### Enhancing User Interaction with Bookmarks

Instead of trying to cram all of the information into the third block, let's make a bookmark! We can make a button that will toggle between the
`Province` and `Improvements` tables. This way the user can choose which data to focus on.

We will create two bookmarks, Province and Improvements. 

- For the Province bookmark, we will hide the Improvements_table.
- For the Improvements bookmark, we will hide the Province_table.

To make the interaction seamless, we will use buttons to toggle between these two bookmarks, allowing users to switch views with ease.

When it is all done, the two panels will look something like this:

![Bookmarks](water_access/visualisation/part7/assets/bookmarks.png)

### Displaying Key Metrics for Decision-Making

To make the report impactful, we include a section showcasing high-impact statistics next to the main heading. These metrics summarize the most critical information at a glance. This is why we calculated measures for water access and no-access rates- to highlight the scope of the problem and the potential impact of our proposed solutions.

The goal of this report is to support in making data-driven decisions. Two fundamental questions need clear answers: How much funding is required? What problem the money will solve?

With this in mind, we added cards to display the following key metrics:

- Total cost of infrastructure upgrades.
- Current percentage of the population with basic access to water.
- Percentage improvement in water access, showing the lives positively impacted.

In [ ]:
# This DAX code to adds the plus sign (+) to the Improvement % card

'''Formatted_improvement = 
    VAR
        ChangePct =
        [Pct_population_without_access]
    RETURN
    IF(
        ChangePct > 0,
            "+" & FORMAT(ChangePct, "0") & "%",
        FORMAT(ChangePct, "0") & "%"
    )'''

Here is the national-level report summary displayed on our dashboard.

![National Report](water_access/visualisation/part7/assets/national_report.png)

### Provincial Leader Dashboard Overview

Our second user is a **provincial leader**. Their user story is: As a leader of a particular province, I want to see data only relevant to my province. I need to understand the state of water access in my province, the scope of work we have to complete, and understand the financial aspects related to the improvements

We will need to show the following data:
1. Total people served for each water source type in a province.
2. Number of water sources, their type, and whether it is rural or urban.
3. Show the relevant stats for towns in that province.
4. Add relevant provincial data. Queues, gender compositions and crime, broken taps by town, etc., in provinces where it is relevant.
5. Summary of improvements and costs.

Having developed the national-level report, the next step is to create similar reports for each province. Once the layout for one province is set up, duplicating the pages and making necessary adjustments for each region becomes much easier. At a minimum, the report should include:

- A breakdown of the budget for each town, detailing both quantity and cost.
- Two visualizations that display the urban/rural split within each province.
- A budget allocation visual to show how resources are distributed across the province.

> **NOTE**: Add drill-through options so we can drill through to the provincial pages from the main page.

By following this structure, we will be able to build out comprehensive provincial reports that align with the national overview while highlighting regional specifics.

![Sokoto Report](water_access/visualisation/part7/assets/sokoto_report.png)

## Summary

While this report isn't perfect, it's more important to get it into the hands of the user first to gauge its usefulness before spending extensive time on optimization. Iteration is key—allowing for continuous improvements over time. For now, once we've made the necessary updates, We will publish the dashboard and inform our directors. They will be using this report to mobilize teams on the ground to address the water crisis with our support. 

It's exciting to see how everything we've worked on is directly contributing to data-driven decision-making that will help solve a real-world issue! It's truly rewarding to look back on the progress we've made—from where we started to where we are now.